# [08] 손실함수 설정하기

데이터로부터의 예측을 바탕으로 라벨과 비교하여 모델을 학습하는 방법을 지도학습(Supervised Learning)이라고 합니다. 지도학습에서는 라벨과 예측을 어떻게 비교할지를 손실함수(Loss Function)을 통해 명확히 정의해야 합니다. 객체인식(Object Detection)에서의 손실함수를 설정하는 것은 다음의 이유로 상당히 까다롭습니다.
- 1) 모델이 아주 많은 수의 Box를 예측하고 있고 이를 Bounding Box, Class 정보와 매칭시켜 Loss를 정해줘야 함.
- 2) Bounding Box를 잘 잡는 (Location) 것과 Class를 잘 예측하는 (Class Score) 두 가지에 대해 조화롭게 학습해야 함.

다행히, 이전 실습에서 만들었던 Prior Box들을 활용하여 문제의 구조를 손실함수를 계산하기 용이하게 정의할 수 있습니다.

In [4]:
from IPython.display import HTML, display

# Image from c231n Lecture Note
display(HTML("<img src='img/[08]loss1.png'>"))

In [5]:
import torch
from torch.utils.data.dataloader import DataLoader
from materials.DetectionNet import DetectionNet, create_prior_boxes
from materials.datasets import PascalVOCDataset
import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = PascalVOCDataset(data_folder='./data/VOC', split='TRAIN')
train_dataloader = DataLoader(dataset=train_dataset, batch_size=8, shuffle=True, collate_fn=train_dataset.collate_fn)
images, boxes, labels, difficulties = next(iter(train_dataloader))

images = images.to(device)
boxes = [b.to(device) for b in boxes]
labels = [l.to(device) for l in labels]

print('\nImage Batch Shape: {}'.format(images.shape))
print('Boxes Batch Length: {}'.format(len(boxes)))
print('Labels Batch Length: {}'.format(len(labels)))

net = DetectionNet(n_classes=20, unfreeze_keys='all').to(device)
pred_locs, pred_scores = net(images)

print('\n>>> Location 예측의 Shape: {}'.format(pred_locs.shape))
print('>>> Clsss Score 예측의 Shape: {}'.format(pred_scores.shape))

prior_boxes = create_prior_boxes()
print('\n>>> Prior Boxes의 Shape: {}'.format(prior_boxes.shape))


Image Batch Shape: torch.Size([8, 3, 300, 300])
Boxes Batch Length: 8
Labels Batch Length: 8
Loaded pretrained weights for efficientnet-b0

>>> Location 예측의 Shape: torch.Size([8, 3106, 4])
>>> Clsss Score 예측의 Shape: torch.Size([8, 3106, 20])

>>> Prior Boxes의 Shape: torch.Size([3106, 4])


-------------------
## [Task 1] GroundTruth Boxes 만들기 + Location Loss
Prior Boxes들에 대해 정답(box 좌표 및 라벨)을 설정하고, 우리가 예측한 Box들이 해당 정답들을 따라가도록 학습하는 방식을 통해 학습을 할 수 있습니다. Prior Box에 대해 1:1로 정답 Box (그리고 Label)을 지정해 주는 것입니다.

- Box 좌표 : 정답 Box 좌표에 대해 Regression
- Box 라벨 : 정답 Label에 대해 Cross-Entropy Loss

### ToDo: `create_prior_boxes` 함수 완성하기

주어진 값들을 바탕으로 GT prior box들을 만들어 봅니다.

### ToDo: Location Loss 계산하기

GT prior box를 사용하여 Location Loss를 계산해 봅니다.

In [6]:
from materials.utils import *

batch_size, num_classes = 8, 20
n_priors = prior_boxes.size(0)

# piror를 cxcy, xy 두 종류의 좌표로 저장합니다.
priors_cxcy = prior_boxes
priors_xy = cxcy_to_xy(prior_boxes)

# Prior Boxes의 개수가 Prediction Boxes의 개수와 같아야 합니다. (3106개)
assert n_priors == pred_locs.size(1) == pred_scores.size(1)

# Prior Boxes에 GroundTruth 정보를 할당하기 위한 Tensor를 만듭니다.
true_locs = torch.zeros((batch_size, n_priors, 4), dtype=torch.float).to(device) # (N, 3106, 4)
true_classes = torch.zeros((batch_size, n_priors), dtype=torch.long).to(device) # (N, 3106)

# Batch 안의 각 Image에 대해
for i in range(batch_size):
    n_objects = boxes[i].size(0) # Label에 존재하는 Object의 개수
    
    # GT Boxes와 Prior Boxes의 Jaccard Overlap을 계산합니다.
    overlap = find_jaccard_overlap(boxes[i], priors_xy) # (n_objects, 3106)
    
    # [ToDo]: 각 prior box에 가장 높은 overlap을 가지는 object에 대한 (overlap값, object idx)를 구합니다.
    overlap_for_each_prior, object_for_each_prior = ?????????????? # (3106) (3106)
    
    # [ToDo]: 각 obejct와 가장 높은 overlap을 가지는 box의 idx를 찾습니다.
    _, prior_for_each_object = ???????????? # (n_objects, )
    
    # 각 object들을 가장 높은 overlap을 가지는 prior에 할당합니다.
    # 이 object들에는 overlap값을 1로 바꿔줍니다.
    object_for_each_prior[prior_for_each_object] = torch.LongTensor(range(n_objects)).to(device)
    overlap_for_each_prior[prior_for_each_object] = 1.
    
    # prior에 대해 class label을 할당합니다.
    label_for_each_prior = labels[i][object_for_each_prior]  # (3106)
    
    # [ToDo] : overlap이 threshold(0.5) 미만이면 라벨을 0(background)으로 설정합니다.
    label_for_each_prior[?????????????????] = ?  # (3106)

    # true class 정보를 저장합니다.
    true_classes[i] = label_for_each_prior

    # regression을 위한 좌표로 바꾸어 true box 좌표 정보를 저장합니다.
    true_locs[i] = cxcy_to_gcxgcy(xy_to_cxcy(boxes[i][object_for_each_prior]), prior_boxes)  # (3106, 4)

# Positivie(class 라벨이 1~20중 하나로 존재), Negative prior(background)에 대한 mask를 얻습니다.
positive_priors = true_classes != 0 # (N, 3106)
negative_priors = true_classes == 0 # (N, 3106)

In [7]:
print('Batch에 있는 8개 이미지 데이터에 대한 Positive Prior의 개수:')
print(positive_priors.sum(dim=1).tolist())

print('\nBatch에 있는 n8개 이미지 데이터에 대한 Negative Prior의 개수:')
print(negative_priors.sum(dim=1).tolist())

Batch에 있는 8개 이미지 데이터에 대한 Positive Prior의 개수:
[28, 18, 13, 29, 12, 25, 4, 15]

Batch에 있는 n8개 이미지 데이터에 대한 Negative Prior의 개수:
[3078, 3088, 3093, 3077, 3094, 3081, 3102, 3091]


In [8]:
import torch.nn as nn

# Location Loss를 L1로 지정합니다.
smooth_l1 = nn.L1Loss()

# [ToDo] : pred_locs와 true_locs의 positivie값들에 대해 Location Loss를 연산합니다. 
loc_loss = smooth_l1(??????????????????, ??????????????????)  # (), scalar

print('Location Loss: %3.4f' % loc_loss.item())

Location Loss: 3.5385


-------------------
## [Task 2] Confidence Loss 만들기
Prior Boxes(Ground Truth)와 Pred Boxes의 라벨을 비교하여 정답 라벨을 맞출 수 있도록 학습하는 Confidence Loss를 만들어 보도록 하겠습니다. 이번에는 Location Loss처럼 Positivive만 비교하지 않고 Negative Prior도 함께 사용하여 Background에 해당하는 Box도 걸러낼 수 있도록 하고 싶습니다. 그러나 Negative Prior의 개수는 Positive보다 훨씬 많기 때문에, 이 중에서 학습에서 효과적으로 활용할 수 있는 Negative Prior들을 `Hard Negative Mining`을 통해 뽑아서 사용합니다.

- Hard Negative Mining
많은 Negative 중에서 어려운 Negative들만을 뽑아서 사용함으로써 Negative에 대한 학습을 효과적으로 수행합니다. 이번에는 Negative Prior중에서 **"Loss가 큰 순서대로 정렬하여 Negative를 Positive의 3배 만큼의 개수만 반영"** 하는 방식으로 Hard Negative Mining을 해보겠습니다.

### ToDo: Hard Negatvie Mining + Confidence Loss 계산하기
- Hard Negative Prior에 대한 Loss만 반영하여 Confidence Loss를 계산합니다.

In [13]:
# CONFIDENCE LOSS를 계산하기 위한 CrossEntropyLoss를 선언합니다.
criterion = nn.CrossEntropyLoss(reduction='none')
n_classes = 20
neg_pos_ratio = 3

# image당 hard_negative로 삼을 prior의 개수를 계산합니다.
n_positives = positive_priors.sum(dim=1)  # (N)
n_hard_negatives = neg_pos_ratio * n_positives  # (N)

# 모든 prior에 대한 loss를 계산합니다.
conf_loss_all = criterion(pred_scores.view(-1, n_classes), true_classes.view(-1))  # (N * 3106)
conf_loss_all = conf_loss_all.view(batch_size, n_priors)  # (N, 3106)

# Negative Prior에 대해 confidence loss를 저장할 tensor를 따로 만듭니다.
conf_loss_neg = conf_loss_all.clone()  # (N, 3106)
conf_loss_neg[positive_priors] = 0.  # (N, 3106), positive priors 는 고려하지 않을 것이므로 0을 할당.

# [ToDo]: Loss를 기준으로 Negatvie prior들을 sorting합니다. 
conf_loss_neg, _ = ?????????????.sort(dim=????, descending=????)  # (N, 3106), 난이도(loss의 크기)에 따라 sorting

# 가장 큰 n_hard_negatives 개의 prior를 뽑습니다.
hardness_ranks = torch.LongTensor(range(n_priors)).unsqueeze(0).expand_as(conf_loss_neg).to(device)  # (N, 3106)
hard_negatives = hardness_ranks < n_hard_negatives.unsqueeze(1)  # (N, 3106)

torch.Size([24848])


### ToDo: Hard Negatvie Mining + Confidence Loss 계산하기
- positivie, negative 각각에 대한 confidence loss를 구하고, 아래의 수식대로 적용해 봅니다.

In [15]:
from IPython.display import HTML, display

# Image from https://arxiv.org/pdf/1512.02325.pdf
display(HTML("<img src='img/[08]loss2.jpg'>"))

In [16]:
# [ToDo]: positivie prior에 대한 cofidence loss를 가져옵니다.
conf_loss_pos = ???????????????????????  # (sum(n_positives))

# [ToDo]: hard negative prior에 대한 confidence loss를 가져옵니다.
conf_loss_hard_neg = ?????????????????  # (sum(n_hard_negatives))

# Confidence Loss를 계산합니다.
conf_loss = (conf_loss_hard_neg.sum() + conf_loss_pos.sum()) / n_positives.sum().float()  # (), scalar

print('Confidence Loss: %3.4f\n' % conf_loss.item())

# Location Loss와 Confidence Loss를 더해 최종 Loss를 만듭니다.
final_loss = loc_loss + conf_loss
print('Final Loss: %3.4f\n' % final_loss.item())

Confidence Loss: 91.3023

Final Loss: 94.8408



-------------------
## MultiBox Loss
아래에는 앞서 구성했던 Location Loss와 Cofidence Loss를 더해 MultiBox Loss로 연산하는 과정을 `nn.Module` class로 표현되어 있습니다. `MultiBoxLoss`는 이미 완성되어 있습니다. class를 통해 연산하고 이전의 결과값과 비교해 봅시다.

### ToDo: MultiboxLoss로 연산하기


In [9]:
import torch
import torch.nn as nn
from materials.utils import *

class MultiBoxLoss(nn.Module):
    def __init__(self, priors_cxcy, threshold=0.5, neg_pos_ratio=3, alpha=1.):
        super(MultiBoxLoss, self).__init__()
        self.priors_cxcy = priors_cxcy
        self.priors_xy = cxcy_to_xy(priors_cxcy)
        self.threshold = threshold
        self.neg_pos_ratio = neg_pos_ratio
        self.alpha = alpha

        self.smooth_l1 = nn.L1Loss()
        self.cross_entropy = nn.CrossEntropyLoss(reduction='none')

    def forward(self, predicted_locs, predicted_scores, boxes, labels):
        batch_size = predicted_locs.size(0)
        n_priors = self.priors_cxcy.size(0)
        n_classes = predicted_scores.size(2)

        assert n_priors == predicted_locs.size(1) == predicted_scores.size(1)

        true_locs = torch.zeros((batch_size, n_priors, 4), dtype=torch.float).to(device)
        true_classes = torch.zeros((batch_size, n_priors), dtype=torch.long).to(device)

        for i in range(batch_size):
            n_objects = boxes[i].size(0)

            overlap = find_jaccard_overlap(boxes[i], self.priors_xy)

            overlap_for_each_prior, object_for_each_prior = overlap.max(dim=0)

            _, prior_for_each_object = overlap.max(dim=1)

            object_for_each_prior[prior_for_each_object] = torch.LongTensor(range(n_objects)).to(device)
            overlap_for_each_prior[prior_for_each_object] = 1.

            label_for_each_prior = labels[i][object_for_each_prior]  # (8732)
            label_for_each_prior[overlap_for_each_prior < self.threshold] = 0  # (8732)

            true_classes[i] = label_for_each_prior
            true_locs[i] = cxcy_to_gcxgcy(xy_to_cxcy(boxes[i][object_for_each_prior]), self.priors_cxcy)  # (8732, 4)

        positive_priors = true_classes != 0

        # LOCALIZATION LOSS
        loc_loss = self.smooth_l1(predicted_locs[positive_priors], true_locs[positive_priors])
        n_positives = positive_priors.sum(dim=1)
        n_hard_negatives = self.neg_pos_ratio * n_positives 
        
        # CONFIDENCE LOSS
        conf_loss_all = self.cross_entropy(predicted_scores.view(-1, n_classes), true_classes.view(-1))
        conf_loss_all = conf_loss_all.view(batch_size, n_priors)

        conf_loss_pos = conf_loss_all[positive_priors]  # (sum(n_positives))

        conf_loss_neg = conf_loss_all.clone()  # (N, 8732)
        conf_loss_neg[positive_priors] = 0.  # (N, 8732), positive priors are ignored (never in top n_hard_negatives)
        conf_loss_neg, _ = conf_loss_neg.sort(dim=1, descending=True)  # (N, 8732), sorted by decreasing hardness
        hardness_ranks = torch.LongTensor(range(n_priors)).unsqueeze(0).expand_as(conf_loss_neg).to(device)  # (N, 8732)
        hard_negatives = hardness_ranks < n_hard_negatives.unsqueeze(1)  # (N, 8732)
        conf_loss_hard_neg = conf_loss_neg[hard_negatives]  # (sum(n_hard_negatives))

        conf_loss = (conf_loss_hard_neg.sum() + conf_loss_pos.sum()) / n_positives.sum().float()  # (), scalar

        # TOTAL LOSS
        return conf_loss + self.alpha * loc_loss


In [10]:
# [ToDo]: MultiBoxLoss 선언하기
criterion = MultiBoxLoss(priors_cxcy=prior_boxes)

In [11]:
boxes = [b.to(device) for b in boxes]
labels = [l.to(device) for l in labels]

# [ToDo]: MultiboxLoss 연산하기
loss = criterion(pred_locs, pred_scores, boxes, labels)

print('Multibox Loss: %3.4f' % loss.item())

Multibox Loss: 94.8408


---------
### <생각해 봅시다>

- Hard Negative Mining의 역할은 무엇인가요?
- Positivie/Negative를 나누는 Overlap의 Threshold는 어떤 의미를 갖나요?
------------